**Import Dependencies**

In [2]:
# Models
from sklearn.svm import LinearSVC #SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
# Other Packages
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
# From PDFInterpreter import both PDFResourceManager and PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
# Import this to raise exception whenever text extraction from PDF is not allowed
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTTextBoxHorizontal, LTFigure, LTChar, LTText, LTAnno
from pdfminer.converter import PDFPageAggregator
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.externals import joblib
import random
import re
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import math
import pickle
import joblib
import json
from gensim.models import KeyedVectors
# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
import warnings
warnings.filterwarnings('ignore')

In [3]:
#define empty lists for columns
pages = []
xcord = []
ycord = []
xcords_first = []
ycords_first = []
font_names = []
content = []
docs = []
objects = []
textboxes = []
i = 0
words = []

password = ""
extracted_text = ""

In [ ]:
ort_col = ['doc', 'Page', 'word']
paper_feature = ['word.is.lower', 'word.is.upper', 'word.is.mixed.case', 'word.is.digit', 'word.contains.digit', 'word.is.special.char','word.len.1', 'word.len.3', 'word.len.5', 'word.len.7', 'word.len.9', 'word.len.11', 'word.len.13', 'word.is.stop']
date_specific_feature = ['word.is.print.date.trigger', 'word.is.revision.date.trigger', 'word.is.valid.date.trigger', 'word.is.oldversion.date.trigger']
new_feature = ['word.is.bold', 'word.is.newline','ycord_average','Xcord_first', 'grid.area_11', 'grid.area_12', 'grid.area_13', 'grid.area_14', 'grid.area_15', 'grid.area_16', 'grid.area_17', 'grid.area_18', 'grid.area_21', 'grid.area_22', 'grid.area_23', 'grid.area_24', 'grid.area_25', 'grid.area_26', 'grid.area_27', 'grid.area_28', 'grid.area_31', 'grid.area_32', 'grid.area_33', 'grid.area_34', 'grid.area_35', 'grid.area_36', 'grid.area_37', 'grid.area_38', 'grid.area_41', 'grid.area_42', 'grid.area_43', 'grid.area_44', 'grid.area_45', 'grid.area_46', 'grid.area_47', 'grid.area_48', 'is.page.1', 'is.page.2', 'is.page.3']

In [29]:
#load existing embeddings
en_model = KeyedVectors.load_word2vec_format('cc.de.300.vec')

Start: Vectors loading


## **1. Data Representation**

In [185]:
df = pd.DataFrame()
pdf_name = '453_sd.pdf'
df = data_representation(pdf_name,df)
df

,doc,Page,Ycord_first,Xcord_first,font_name,Object,Textbox,word,special_char
0,453_sd.pdf,1,788.4084,177.74000,Arial-BoldMT,1,1,Lenor,0
1,453_sd.pdf,1,788.4084,226.56164,Arial-BoldMT,1,1,3in1,0
2,453_sd.pdf,1,788.4084,262.99220,Arial-BoldMT,1,1,Pods,0
3,453_sd.pdf,1,788.4084,306.61088,Arial-BoldMT,1,1,Vollwaschmittel,0
4,453_sd.pdf,1,788.4084,431.97668,Arial-BoldMT,1,1,Seidene,0
6,453_sd.pdf,1,769.9284,177.74000,Arial-BoldMT,1,2,Orchidee,0
8,453_sd.pdf,1,756.1200,177.74000,ArialMT,1,3,Sicherheitsdatenblatt,0
10,453_sd.pdf,1,747.9184,177.74000,ArialMT,1,4,gemäß,0
11,453_sd.pdf,1,747.9184,201.38312,ArialMT,1,4,Verordnung,0
13,453_sd.pdf,1,747.9184,239.87888,ArialMT,1,4,(,1


In [186]:
df = ycord_average(df)
df

,doc,Page,Ycord_first,Xcord_first,font_name,Object,Textbox,word,special_char,ycord_average
0,453_sd.pdf,1,788.4084,177.74000,Arial-BoldMT,1,1,Lenor,0,788.4084
1,453_sd.pdf,1,788.4084,226.56164,Arial-BoldMT,1,1,3in1,0,788.4084
2,453_sd.pdf,1,788.4084,262.99220,Arial-BoldMT,1,1,Pods,0,788.4084
3,453_sd.pdf,1,788.4084,306.61088,Arial-BoldMT,1,1,Vollwaschmittel,0,788.4084
4,453_sd.pdf,1,788.4084,431.97668,Arial-BoldMT,1,1,Seidene,0,788.4084
5,453_sd.pdf,1,769.9284,177.74000,Arial-BoldMT,1,2,Orchidee,0,769.9284
6,453_sd.pdf,1,756.1200,177.74000,ArialMT,1,3,Sicherheitsdatenblatt,0,756.1200
7,453_sd.pdf,1,747.9184,177.74000,ArialMT,1,4,gemäß,0,747.9184
8,453_sd.pdf,1,747.9184,201.38312,ArialMT,1,4,Verordnung,0,747.9184
9,453_sd.pdf,1,747.9184,239.87888,ArialMT,1,4,(,1,747.9184


## **2. Feature Engineering**

In [187]:
df = feature_engineering(df)
df[ort_col+paper_feature+new_feature]

,doc,Page,word,grid.area_11,grid.area_12,grid.area_13,grid.area_14,grid.area_15,grid.area_16,grid.area_17,grid.area_18,grid.area_21,grid.area_22,grid.area_23,grid.area_24,grid.area_25,grid.area_26,grid.area_27,grid.area_28,grid.area_31,grid.area_32,grid.area_33,grid.area_34,grid.area_35,grid.area_36,grid.area_37,grid.area_38,grid.area_41,grid.area_42,grid.area_43,grid.area_44,grid.area_45,grid.area_46,grid.area_47,grid.area_48,word.is.lower,word.is.upper,word.is.mixed.case,word.is.bold,word.is.digit,word.contains.digit,word.is.special.char,word.len.1,word.len.3,word.len.5,word.len.7,word.len.9,word.len.11,word.len.13,word.is.stop,word.is.print.date.trigger,word.is.revision.date.trigger,word.is.oldversion.date.trigger,word.is.valid.date.trigger,grid.area,is.page.1,is.page.2,is.page.3,word.is.newline,ycord_average,Xcord_first
0,453_sd.pdf,1,Lenor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,28,1,0,0,1,788.4084,177.74000
1,453_sd.pdf,1,3in1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,28,1,0,0,1,788.4084,226.56164
2,453_sd.pdf,1,Pods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,1,0,0,0,788.4084,262.99220
3,453_sd.pdf,1,Vollwaschmittel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,38,1,0,0,0,788.4084,306.61088
4,453_sd.pdf,1,Seidene,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,48,1,0,0,0,788.4084,431.97668
5,453_sd.pdf,1,Orchidee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,28,1,0,0,1,769.9284,177.74000
6,453_sd.pdf,1,Sicherheitsdatenblatt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,28,1,0,0,1,756.1200,177.74000
7,453_sd.pdf,1,gemäß,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,28,1,0,0,1,747.9184,177.74000
8,453_sd.pdf,1,Verordnung,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,28,1,0,0,0,747.9184,201.38312
9,453_sd.pdf,1,(,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,28,1,0,0,0,747.9184,239.87888


In [188]:
#Create word embeddings
df = word_embedding(df)
df

In [189]:
# Create sliding window for features and word embeddings
df = sliding_window(df)

## **2. ML Models**

In [190]:
predict_labels(df)

C:\Users\merzg\Anaconda3\envs\aiss_py37\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\merzg\Anaconda3\envs\aiss_py37\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.21.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


   ************************** Verordnung **************************

['2015/830', '2015/830']


   ************************** Versionsnummer **************************

['1.0']


   ************************** Chapter **************************

['ABSCHNITT', '1', ':', 'Bezeichnung', 'des', 'Stoffs', 'beziehungsweise', 'des', 'Gemischs', 'und', 'des', 'Unternehmens', 'ABSCHNITT', '2', ':', 'Mögliche', 'Gefahren', 'ABSCHNITT', '3', ':', 'Zusammensetzung', '/', 'Angaben', 'zu', 'Bestandteilen', 'ABSCHNITT', '4', ':', 'Erste-Hilfe-Maßnahmen', 'ABSCHNITT', '5', ':', 'Maßnahmen', 'zur', 'Brandbekämpfung', 'ABSCHNITT', '6', ':', 'Maßnahmen', 'bei', 'unbeabsichtigter', 'Freisetzung', 'ABSCHNITT', '7', ':', 'Handhabung', 'und', 'Lagerung', 'ABSCHNITT', '8', ':', 'Begrenzung', 'und', 'Überwachung', 'der', 'Exposition', '/', 'Persönliche', 'Schutzausrüstungen']


   ************************** Subchapter **************************

['1.1.', 'Produktidentifikator', '1.2.', 'Relevante', 'identifizie

## **1. Data Representation**

In [8]:
#function for checking if token contains a number
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [9]:
def parse_words(lt_objs, pagenum, doc):
        objectnum = 1
        #iterate through Textboxes
        for obj in lt_objs:
                textboxnum = 1
                if isinstance(obj, LTTextBoxHorizontal):
                #iterate through TextLines
                        for o in obj._objs:
                                if isinstance(o, LTTextLine):
                                        text=o.get_text()
                                        #print(text)
                                        if text.strip():
                                                word = ''
                                                #iterate through characters
                                                for c in  o._objs:
                                                        #if it is a character
                                                        if isinstance(c, LTChar):
                                                                #detect special character at end of words
                                                                if c.get_text() in (')',':','%',';','('):
                                                                    words.extend([word, str(c.get_text())])
                                                                    pages.extend([pagenum, pagenum])
                                                                    xcords_first.extend([xcord_first,c.bbox[0]])
                                                                    ycords_first.extend([ycord_first,c.bbox[1]])
                                                                    docs.extend([doc, doc])
                                                                    font_names.extend([fontname,c.fontname])
                                                                    objects.extend([objectnum, objectnum])
                                                                    textboxes.extend([textboxnum, textboxnum])
                                                                    word = ''
                                                                # if / in between of numbers --> not a new token
                                                                elif c.get_text() in ('/') and not word.isdigit() and not word == 'g':
                                                                    words.extend([word, str(c.get_text())])
                                                                    pages.extend([pagenum, pagenum])
                                                                    xcords_first.extend([xcord_first,c.bbox[0]])
                                                                    ycords_first.extend([ycord_first,c.bbox[1]])
                                                                    docs.extend([doc, doc])
                                                                    font_names.extend([fontname,c.fontname])
                                                                    objects.extend([objectnum, objectnum])
                                                                    textboxes.extend([textboxnum, textboxnum])
                                                                    word = ''
                                                                # if , in between of numbers --> not a new token
                                                                elif c.get_text() in (',') and not word.isdigit():
                                                                    words.extend([word, str(c.get_text())])
                                                                    pages.extend([pagenum, pagenum])
                                                                    xcords_first.extend([xcord_first,c.bbox[0]])
                                                                    ycords_first.extend([ycord_first,c.bbox[1]])
                                                                    font_names.extend([fontname,c.fontname])
                                                                    docs.extend([doc, doc])
                                                                    objects.extend([objectnum, objectnum])
                                                                    textboxes.extend([textboxnum, textboxnum])
                                                                    word = ''
                                                                
                                                                # take 'n.v' as an exception and check that . is not in between numbers or dates
                                                                elif c.get_text() in ('.') and not hasNumbers(word) and not word == 'n' and '@' not in word:
                                                                    words.extend([word, str(c.get_text())])
                                                                    pages.extend([pagenum, pagenum])
                                                                    xcords_first.extend([xcord_first,c.bbox[0]])
                                                                    ycords_first.extend([ycord_first,c.bbox[1]])
                                                                    font_names.extend([fontname,c.fontname])
                                                                    docs.extend([doc, doc])
                                                                    objects.extend([objectnum, objectnum])
                                                                    textboxes.extend([textboxnum, textboxnum])
                                                                    word = ''
                                                                    
                                                                else:
                                                                    #append until space
                                                                    word += str(c.get_text())
                                                                    #remember coords if its first char of word
                                                                    if len(word) == 1:
                                                                            xcord_first = c.bbox[0]
                                                                            ycord_first = c.bbox[1]
                                                                            fontname = c.fontname
                                                                    # if space and previous token was not space: append word to list (without the space) and start new word
                                                                    if c.get_text() == ' ':
                                                                            words.append(word[:-1])
                                                                            pages.append(pagenum)
                                                                            xcords_first.append(xcord_first)
                                                                            ycords_first.append(ycord_first)
                                                                            font_names.append(fontname)
                                                                            docs.append(doc)
                                                                            objects.append(objectnum)
                                                                            textboxes.append(textboxnum)
                                                                            word = ''
                                                        #if it is a new line and word is not empty: append word to list and start new word
                                                        if isinstance(c, LTAnno):
                                                                words.append(word)
                                                                pages.append(pagenum)
                                                                xcords_first.append(xcord_first)
                                                                ycords_first.append(ycord_first)
                                                                font_names.append(fontname)
                                                                docs.append(doc)
                                                                objects.append(objectnum)
                                                                textboxes.append(textboxnum)
                                                                word = ''
             
                                textboxnum = textboxnum + 1
                                                                                         
                # if it's a container, recurse
                elif isinstance(obj, LTFigure):
                        parse_words(obj._objs, pagenum, doc)

                objectnum = objectnum + 1

In [125]:
def data_representation(pdf_name, df):
    
    #read PDF file
    doc = pdf_name
    fp = open(doc, 'rb')

    # Create parser object to parse the pdf content
    parser = PDFParser(fp)

    # Store the parsed content in PDFDocument object
    document = PDFDocument(parser, password)

    # Create PDFResourceManager object that stores shared resources such as fonts or images
    rsrcmgr = PDFResourceManager()

    # set parameters for analysis
    laparams = LAParams()

    # Create a PDFDevice object which translates interpreted information into desired format
    # Device needs to be connected to resource manager to store shared resources
    device = PDFDevice(rsrcmgr)
    # Extract the decive to page aggregator to get LT object elements
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)

    # Create interpreter object to process page content from PDFDocument
    # Interpreter needs to be connected to resource manager for shared resources and device 
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # process only first 3 pages 
    for p ,page in enumerate(PDFPage.create_pages(document)):
        pagenum = p + 1        
        if pagenum == 4:
            break
        # As the interpreter processes the page stored in PDFDocument object
        interpreter.process_page(page)
        # The device renders the layout from interpreter
        layout = device.get_result()

        # call function to parse on character level and identify tokens
        parse_words(layout._objs, pagenum, doc)


    #close the pdf file
    fp.close()
    
    #save data in dataframe
    
    #creat empty dataframe
    df = pd.DataFrame( 
        {
         'doc': docs,
         'Page': pages,
         'Ycord_first': ycords_first,
         'Xcord_first': xcords_first,
         'font_name': font_names,
         'Object': objects,
         'Textbox': textboxes,
         'word': words
        })

    df = df.sort_values(['doc','Page','Ycord_first','Xcord_first'],ascending=[True,True,False,True])

    #delete empty tokens
    df = df[ df["word"] != ""]

    #identify special characters
    df['special_char'] = df['word'].apply(lambda x: 1 if (x in ('"*"', 'ꞏ') or (len(x) == 1 and x.isalnum () == False)) else 0 )

    return df

In [11]:
#sort by average of y-coordinate within the same line

def ycord_average(df):

    yc = df['Ycord_first']
    yc_new = []


    counter = 0
    first = True
    for index, i in enumerate(yc):
        #very first row
        if first == True:
            avg = i
            counter += 1 
        elif abs(avg - i) <= 5:
            avg = avg - ((avg - i)/(counter+1))
            counter += 1
            #very last row
            if index == (len(yc)-1):
                yc_new += counter * [avg]
        elif abs(avg - i) > 5:
            yc_new += counter * [avg]
            avg = i
            counter = 1


        first = False


    df['ycord_average'] = yc_new


    df = df.sort_values(['doc','Page','ycord_average','Xcord_first'],ascending=[True,True,False,True])

    df.reset_index(inplace=True, drop = True)
    
    return df

## **2. Feature Engineering**

In [12]:
def feature_engineering(df):

    words = list(df['word'])

    #set german stopwords
    stopWords = set(stopwords.words('german'))

    # define own special character set 
    special_chars = ('.',',','(', ')', '–', '[', '·','{', '}', ']', ':', ';', "'", '"','?', '/', '*','!', '@', '#', '&', '"*"', '`', '~', '$', '^', '+', '=', '<', '>','%')

    #prepare ranges for x-y-coordinate grid classification
    num_rows = 8
    num_cols = 4

    x_width = int(max(df['Xcord_first'])) / num_cols
    y_width = int(max(df['ycord_average'])) / num_rows


    features = {

    # meta information: style of characters

        'word.is.lower': [],
        'word.is.upper': [],
        'word.is.mixed.case': [],
        'word.is.bold':[],


    # meta information: type of characters

        'word.is.digit': [],
        'word.contains.digit':[],
        'word.is.special.char' :[],


    # length of words

        'word.len.1':[],
        'word.len.3':[],
        'word.len.5':[],
        'word.len.7':[],
        'word.len.9':[],
        'word.len.11':[],
        'word.len.13':[],

    # Semantic

        #'word.pos':[],
        'word.is.stop':[],
        'word.is.print.date.trigger':[],
        'word.is.revision.date.trigger':[],
        'word.is.oldversion.date.trigger':[],
        'word.is.valid.date.trigger':[],


        # Graphical Information / Location of word(token)
        'grid.area':[],

        'is.page.1':[],
        'is.page.2':[],
        'is.page.3':[],

        'word.is.newline': []    
        }

    '''
    # Graphical Information / Location of word(token)


        'word.is.koord.1':[],
        'word.is.koord.2':[],
        'word.is.koord.3':[],
        'word.is.koord.4':[],
        'word.is.koord.5':[],
        'word.is.koord.6':[],
        'word.is.koord.7':[],
        'word.is.koord.8':[],

        'is.page.1':[],
        'is.page.2':[],
        'is.page.3':[],

        'word.is.newline': []    
    '''


    for i, w in enumerate(words):

        '''
        meta information: style of characters
        '''
        features['word.is.lower'].append(str(w).islower())
        features['word.is.upper'].append(str(w).isupper())

        if not str(w).islower() and not str(w).isupper():
            features['word.is.mixed.case'].append(1)
        else:
            features['word.is.mixed.case'].append(0)

        if 'bold' in str(df.loc[i,'font_name']).lower():
            features['word.is.bold'].append(1)
        else:
            features['word.is.bold'].append(0)


        '''
        meta information: type of characters
        '''

        features['word.is.digit'].append(str(w).isdigit())
        if any(x.isdigit() for x in str(w)):
            features['word.contains.digit'].append(1)
        else:
            features['word.contains.digit'].append(0)

        if str(w) in special_chars:
            features['word.is.special.char'].append(1)
        else:
            features['word.is.special.char'].append(0)
            

        '''
        length of words
        '''
        if len(str(w)) > 12:
            features['word.len.13'].append(1)
            for l in (1,3,5,7,9,11):
                features['word.len.' + str(l)].append(0)
        elif len(str(w)) > 10:
            features['word.len.11'].append(1)
            for l in (1,3,5,7,9,13):
                features['word.len.' + str(l)].append(0)
        elif len(str(w)) > 8:
            features['word.len.9'].append(1)
            for l in (1,3,5,7,11,13):
                features['word.len.' + str(l)].append(0)
        elif len(str(w)) > 6:
            features['word.len.7'].append(1)
            for l in (1,3,5,9,11,13):
                features['word.len.' + str(l)].append(0)
        elif len(str(w)) > 4:
            features['word.len.5'].append(1)
            for l in (1,3,7,9,11,13):
                features['word.len.' + str(l)].append(0)
        elif len(str(w)) > 2:
            features['word.len.3'].append(1)
            for l in (1,5,7,9,11,13):
                features['word.len.' + str(l)].append(0)
        else:
            features['word.len.1'].append(1)
            for l in (3,5,7,9,11,13):
                features['word.len.' + str(l)].append(0)

        '''
        Semantic
        '''



        if str(w) in stopWords:
            features['word.is.stop'].append(1)
        else:
            features['word.is.stop'].append(0)

        if str(w).lower() in ['druck', 'ausgabe', 'ausstellung', 'erstellung', 'sd-datum', 'erstellt', 'ausgestellt']:
            features['word.is.print.date.trigger'].append(1)
        else:
            features['word.is.print.date.trigger'].append(0)

        if str(w).lower() in ['überarbeit', 'änderung', 'revision', 'bearbeitung', 'quick-fds']:
            features['word.is.revision.date.trigger'].append(1)
        else:
            features['word.is.revision.date.trigger'].append(0)

        if str(w).lower() in  ['ersetzt', 'ersatz', 'fassung', 'letzten']:
            features['word.is.oldversion.date.trigger'].append(1)
        else:
            features['word.is.oldversion.date.trigger'].append(0)

        if str(w).lower() in ['kraft', 'freigabe']:
            features['word.is.valid.date.trigger'].append(1)
        else:
            features['word.is.valid.date.trigger'].append(0)


        # Graphical Information / Location of word(token)

        for r in reversed(range(num_rows)):
            if df.loc[i,'ycord_average'] > r*y_width:
                y_cluster = r+1
                break
            else:
                y_cluster = 1
        for c in reversed(range(num_cols)):
            if df.loc[i,'Xcord_first'] > c*x_width:
                x_cluster = c+1
                break
            else:
                x_cluster = 1

        cluster = str(str(x_cluster) + str(y_cluster))
        features['grid.area'].append(cluster)

        if df.loc[i,'Page'] == 1:
            features['is.page.1'].append(1)
            features['is.page.2'].append(0)
            features['is.page.3'].append(0)
        elif df.loc[i,'Page'] == 2:
            features['is.page.1'].append(0)
            features['is.page.2'].append(1)
            features['is.page.3'].append(0)
        elif df.loc[i,'Page'] == 3:
            features['is.page.1'].append(0)
            features['is.page.2'].append(0)
            features['is.page.3'].append(1)

        if i > 1:
            if df.loc[i,'ycord_average'] != df.loc[i-1,'ycord_average']:
                features['word.is.newline'].append(1)
            else:
                features['word.is.newline'].append(0)
        else:
            features['word.is.newline'].append(1)


    features = pd.DataFrame(features)

    orientation_col = df[['doc', 'Page', 'word']]

    df = pd.concat((orientation_col, features, df[['ycord_average', 'Xcord_first']]), axis=1, sort=False)
    
    #encode columns

    #one-hot grid areas
    df.loc [:,'grid.area'] = pd.Categorical(df['grid.area'])
    grid_dummies = pd.get_dummies(df['grid.area'], prefix = 'grid.area') 

    #first columns as id, not part of the model 
    orientation_col = df.loc[:, 'doc':'word']

    #select feature columns
    features = df.loc[:, 'word.is.lower':'Xcord_first']
    #change remaining features from True/False to 0/1
    features.loc[:,'word.is.lower'] = features['word.is.lower'].astype(int)
    features.loc[:,'word.is.upper'] = features['word.is.upper'].astype(int)
    features.loc[:,'word.is.digit'] = features['word.is.digit'].astype(int)



    df = pd.concat((orientation_col, grid_dummies, features), axis=1, sort=False)
    
    return df


In [28]:
# create word embeddings

def word_embedding(df):

    #preprocess words
    word_emb_input = pd.DataFrame(df.loc[:,'word'].str.lower())
    word_emb_input = pd.DataFrame(word_emb_input)
    word_emb_input = word_emb_input.astype(str)
    word_emb_input['preprocessed'] = np.nan
    word_emb_input ['preprocessed'] = [re.sub('\d', 'D', x) for x in word_emb_input['word'].tolist()]
    word_emb_input = word_emb_input.drop(['word'],1)

    #create embeddings

    #calculate embeddings for every word
    missing=[0]*300
    def fun(key):
        try:
            return(en_model[key])
        except:
            return(missing)
    word_emb_input['vector'] = word_emb_input['preprocessed'].apply(fun)
    word_emb = pd.DataFrame(word_emb_input['vector'].values.tolist())

    #concat data
    emb_data = pd.concat((df, word_emb), axis=1, sort=False)
    return emb_data



In [14]:
def sliding_window(df):

    # create sliding window

    #Feature Groups
    ort_col = ['doc', 'Page', 'word']
    paper_feature = ['word.is.lower', 'word.is.upper', 'word.is.mixed.case', 'word.is.digit', 'word.contains.digit', 'word.is.special.char','word.len.1', 'word.len.3', 'word.len.5', 'word.len.7', 'word.len.9', 'word.len.11', 'word.len.13', 'word.is.stop']
    date_specific_feature = ['word.is.print.date.trigger', 'word.is.revision.date.trigger', 'word.is.valid.date.trigger', 'word.is.oldversion.date.trigger']
    new_feature = ['word.is.bold', 'word.is.newline','ycord_average','Xcord_first', 'grid.area_11', 'grid.area_12', 'grid.area_13', 'grid.area_14', 'grid.area_15', 'grid.area_16', 'grid.area_17', 'grid.area_18', 'grid.area_21', 'grid.area_22', 'grid.area_23', 'grid.area_24', 'grid.area_25', 'grid.area_26', 'grid.area_27', 'grid.area_28', 'grid.area_31', 'grid.area_32', 'grid.area_33', 'grid.area_34', 'grid.area_35', 'grid.area_36', 'grid.area_37', 'grid.area_38', 'grid.area_41', 'grid.area_42', 'grid.area_43', 'grid.area_44', 'grid.area_45', 'grid.area_46', 'grid.area_47', 'grid.area_48', 'is.page.1', 'is.page.2', 'is.page.3']

    data_ord = df[ort_col]

    columns = [paper_feature, date_specific_feature, new_feature]

    #create window for all features except word embedding
    window_size_feat = 13
    #copies the previous and following features for every token in a given window
    for col in range (len(columns)):
        sel_col = df[columns[col]]
        data_ord = pd.concat([data_ord, sel_col], axis=1, sort=False)
        for i in range (1, math.ceil(window_size_feat/2)):
            data_pre = sel_col.shift(i).add_prefix ('-' + str(i) + '_')
            data_suc = sel_col.shift(-i).add_prefix ('+' + str(i) + '_')
            data_ord = pd.concat([data_ord, data_pre, data_suc], axis=1, sort=False)

    df.columns = df.columns.astype(str)
    
    emd_col = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
'23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
'66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
'163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291',
'292', '293', '294', '295', '296', '297', '298','299']

    
    #create window for word embedding
    window_size_emb = 3
    sel_col = df.loc[:,emd_col]
    data_ord = pd.concat([data_ord, sel_col], axis=1, sort=False)
    for i in range (1, math.ceil(window_size_emb/2)):
        data_pre = sel_col.shift(i).add_prefix ('-' + str(i) + '_')
        data_suc = sel_col.shift(-i).add_prefix ('+' + str(i) + '_')
        data_ord = pd.concat([data_ord, data_pre, data_suc], axis=1, sort=False)
        
    return data_ord

## **ML Model**

In [109]:
def predict_labels(df):
    
  
    # load the model from disk
    ml_model = joblib.load('Own+Paper+Wordembedding_label_group.pkl')

    #prepare data
    df = df.fillna(0)

    df1 = df.loc[:,'word.is.bold':'+6_is.page.2'] #'+6_is.page.3'
    df2 = df.loc[:,'word.is.lower':'+6_word.len.13']#+6_word.is.stop
    df3 = df.loc[:,'0':'+1_298']#+1_299

    X = pd.concat([df1, df2, df3], axis=1, sort=False)
    
    #predict labels
    y_pred = ml_model.predict(X)


    df['label_pred'] = y_pred

    df_pred = df.loc[:,['word','label_pred']]

    version = df_pred[df_pred['label_pred'] == 8]
    version = list(version['word'])

    directive = df_pred[df_pred['label_pred'] == 3]
    directive = list(directive['word'])

    signal = df_pred[df_pred['label_pred'] == 4]
    signal = list(signal['word'])

    chapter = df_pred[df_pred['label_pred'] == 1]
    chapter = list(chapter['word'])

    subchapter = df_pred[df_pred['label_pred'] == 5]
    subchapter = list(subchapter['word'])

    usecase_con = df_pred[df_pred['label_pred'] == 6]
    usecase_con = list(usecase_con['word'])

    usecase_pro = df_pred[df_pred['label_pred'] == 7]
    usecase_pro = list(usecase_pro['word'])

    '''
    dict = {
        'directive': directive,
        'signal': signal,
        'version-nr': version,  
        'chapter': chapter,
        'subchapter': subchapter,
        'should be used for': usecase_pro,
        'should not be used for': usecase_con
    }
    '''

    print ('   ************************** Verordnung **************************\n')
    print(directive)
    print('\n')

    print ('   ************************** Versionsnummer **************************\n')
    print(version)
    print('\n')

    print ('   ************************** Chapter **************************\n')
    print(chapter)
    print('\n')

    print ('   ************************** Subchapter **************************\n')
    print(subchapter)
    print('\n')

    print ('   ************************** Signalwort **************************\n')
    print(signal)
    print('\n')

    print ('   ************************** Relevante Verwendung **************************\n')
    print(usecase_pro)
    print('\n')

    print ('   ************************** Keine Verwendung **************************\n')
    print(usecase_con)
    print('\n')

    #print(json.dumps(dict, indent=2))